In [71]:
# Set up the environment 
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [72]:
# Clone the ResNet implementation we are going to use
# Clone the ResNet implementation we are going to use
!git clone https://github.com/geos98/CS294-082.git
%cd CS294-082
!git clone https://github.com/akamaster/pytorch_resnet_cifar10

fatal: destination path 'pytorch_resnet_cifar10' already exists and is not an empty directory.


In [74]:
from pytorch_resnet_cifar10.resnet import resnet20

state_dict_path = 'resnet_20_pretrain.pt'
state_dict = torch.load(state_dict_path, map_location=device)

model = resnet20()
miss = model.load_state_dict(state_dict)

print(miss)

model.to(device)
model.eval()


<All keys matched successfully>


ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [75]:
# Create datasets
import torchvision.transforms as transforms
import torchvision

from torch.utils.data import DataLoader

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize,
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

# Train data with test set trainsform (i.e., no crop, no flip)
traintestset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=test_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=test_transform)

train_loader = DataLoader(trainset, batch_size=128, shuffle=True, pin_memory=True)
test_loader = DataLoader(testset, batch_size=128, shuffle=False, pin_memory=True)
traintest_loader = DataLoader(traintestset, batch_size=128, shuffle=False, pin_memory=True)


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [77]:
# Evaluation on accuracy
from tqdm import tqdm


with torch.no_grad():
    model.eval()

    correct_test = 0
    total_test = 0
    # Accuracy on test set
    for p, l in tqdm(test_loader):
        p = p.to(device)
        out = model(p)
        _, predicted = out.max(1)

        correct_test += predicted.eq(l).sum().item()
        total_test += l.size(0)
    print('Testing Set: Predicted {} instances out of {}'.format(correct_test, total_test))

    correct_train = 0
    total_train = 0
    # Accuracy on test set
    for p, l in tqdm(traintest_loader):
        p = p.to(device)
        out = model(p)
        _, predicted = out.max(1)

        correct_train += predicted.eq(l).sum().item()
        total_train += l.size(0)
    print('Trainning Set: Predicted {} instances out of {}'.format(correct_train, total_train))

100%|██████████| 79/79 [00:16<00:00,  4.86it/s]


Testing Set: Predicted 9177 instances out of 10000


100%|██████████| 391/391 [01:22<00:00,  4.75it/s]

Trainning Set: Predicted 49861 instances out of 50000


## Q1: What is the variable the machine learner is supposed to predict? How accurate is the labeling? What is the annotator agreement (measured)?
The CIFAR-10 dataset consists of 60,000 images of 10 classes which are  `airplane`, `automobile`, `bird`, `cat`, `deer`, `dog`, `frog`, `horse`, `ship` and `truck`. The machine learner should take-in an image and predict what is the object in the image by outputing one of the 10 classes listed above.

There are a total of 22 known and validated label error exists in this dataset[1]. The label error was found algorithmically and further verified by human. This is 99.96% label accuracy. For the purpose of this project, we will ignore this error and assume the labelling is correct. 

Researchers at the University of Toronto paid human labler to label the dataset[2]. Each labelling is done by one human labler and verified by the one of the researchers after initial labelling. The in-accuracy could be caused by human error due to the large workload needed to label this dataset.

## Q2: What is the required accuracy metric for success? How much data do we have to train the prediction of the variable? Are the classes balanced? How many modalities could be exploited in the data? Is there temporal information? How much noise are we expecting? Do we expect bias?

Since this dataset serves as a benchmark towards the machine learner's ability to classify subjects in the image, the required accuracy metric is higher the better. However, there were no real-world implication for the success at this time.

We have 60,000 images, splited into a trainning set of 50,000 images and testing set of 10,000 images. 

The classes are perfectly balanced in both trainning set and testing set. Meaning that in total there are 6,000 images of each class.

There is only one modality, the visual, that we can exploit. 

There are no temporal information. 

Since every sample is an image with a background, and we are only interested in the main subject, this means that all the background portion of the image is effectively noise for our purpose. Therefore, we need to apply strategies like convolution to extract important features.

We expect the machine learner to focus on the center of the image since that is the place where most subject would exists in the image.

## Q3: What is the MEC of the data? What is the expected MEC for a neural net?

Since we only care about the MEC of the data after the "compression", we will create a CSV file of data after compression

In [78]:
# Generate .csv from pre-trained ResNet-18 (only using the conv layer to extract deepfeature)
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from itertools import chain

# Get pre-trained conv output
def run_conv(x):
    out = F.relu(model.bn1(model.conv1(x)))
    out = model.layer1(out)
    out = model.layer2(out)
    out = model.layer3(out)
    out = F.avg_pool2d(out, out.size()[3])
    out = out.view(out.size(0), -1)

    return out

with open('compressed.csv', 'w+') as fout:
    with torch.no_grad():
        model.eval()
        total_len = len(traintestset) + len(testset)
        for p, l in tqdm(chain(traintestset, testset), total = total_len):
            p = torch.unsqueeze(p, 0).to(device)
            conv_data = run_conv(p)
            np_data = conv_data.flatten().cpu().numpy()
            CSV_str = ','.join(['%.15f' % num for num in np_data])
            CSV_str += ',{}'.format(l)
            fout.write(CSV_str + '\n')
            

100%|██████████| 60000/60000 [03:27<00:00, 288.82it/s]


We can either run Brainome on the dataset

In [ ]:
# Install Brainome for analysis
#%pip install brainome
#!brainome login
#!brainome compressed.csv -y -headerless -f NN 

Or we can run our own analysis

In [81]:
import pandas as pd
import math

from IPython.display import display

# Implement Alogrithm 2 in the book
df = pd.read_csv('compressed.csv', header=None)
dim = df.shape[1] - 1

df = pd.DataFrame({'sum': df.iloc[:, :-1].sum(axis = 1), 'label': df.iloc[:, 64]})

# Drop the raw pixels
df.sort_values('sum')
display(df.head(10))

thr = (df['label'].diff() != 0).sum()
print('Total Threshold = ', thr)

min_thr = math.log2(thr + 1)
mec = (min_thr * (dim + 1)) + (min_thr + 1)
print('MEC = ', mec, ' bits')

,sum,label
0,43.632485,6
1,65.542533,9
2,51.231566,9
3,40.198811,4
4,58.481588,1
5,49.246642,1
6,59.517407,2
7,49.590992,7
8,46.345304,8
9,50.236550,3


Total Threshold =  54082
MEC =  1038.7105789672364  bits


As calculated above, we find that the MEC of the data is around 1038.7 bits.


The decision layer in ResNet-20 is a simple fully connected layer with 64 input and 10 output. That is, 10 neurons each with 64 input.Now, each individual neuron therefore have parameter count of 64 + 1 = 65 and the MEC of the network is  
65 * 10 = 650 bits

## Q4: What is the expected generalization in bits/bit and as a consequence the average resilience in dB? Is the resilience enough for the task? How bad can adversarial examples be? Do we expect data drift?

In [83]:
# Test the data on pre-trained decision layer
from torch.utils.data.dataset import Dataset
class CompressedCifar10(Dataset):
    def __init__(self):
        df = pd.read_csv('compressed.csv', header=None)

        x = df.iloc[:, 0:64].values
        y = df.iloc[:, 64].values

        self.x_data = torch.tensor(x, dtype=torch.float32, device=device)
        self.y_data = torch.tensor(y, dtype=torch.int32, device=device)
    
    def __len__(self):
        return len(self.y_data)
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

total_correct = 0
with torch.no_grad():
    dataset = CompressedCifar10()

    for x, y in tqdm(dataset):
        preidct = model.linear(x)
        predict = torch.argmax(preidct)

        if predict == y:
            total_correct += 1

print('Correctly predicted instances: ', total_correct)


100%|██████████| 60000/60000 [00:00<00:00, 70162.33it/s]

Correctly predicted instances:  59038


Therefore, combine MEC = 650 bit, we get the Generalization is

$$G = \frac{59038  \log_2{10}}{MEC} = 188.81 \frac{\text{bits}}{\text{bit}}$$

Which would then traslate to resilience $R = 20 * \log_{10}G = 20 * \log_{10} 188.81 = 45.52 \text{dB}$

Since this is a benchmark for machine learning models, we don't expect adversarial examples nor do we expect data drift. The data is as-is, nothing new would be added.

## Reference
[1] https://arxiv.org/abs/2103.14749  
[2] https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf